In [2]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd
import muon as mu
import scanpy as sc
import scirpy as ir
np.random.seed(42)
import random
random.seed(42)

In [3]:
mdata = mu.read("/ix1/ylee/Yifan_Zhang/Code_data/data_EAE/anndata/all_common_DE2500_metaData.h5mu")
mdata

MuData object with n_obs × n_vars = 77500 × 2500
  obs:	'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'HTO_maxID', 'HTO_secondID', 'HTO_margin', 'HTO_classification', 'HTO_classification.global', 'hash.ID', 'percent.mt', 'integrated_nn_res.2', 'seurat_clusters', 'level_2_clusters', 'manual_cell_type', 'run_num', 'sample', 'VJ_1_cdr3_aa', 'VJ_1_v_call', 'VJ_1_j_call', 'VDJ_1_cdr3_aa', 'VDJ_1_v_call', 'VDJ_1_j_call', 'celltype_sample', 'cloned', 'in_two_tissue', 'TCRdist_MOG'
  2 modalities
    gex:	77500 x 2500
      obs:	'mouse_id', 'date', 'tissue', 'sample', 'mouse_BC', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_genes', 'n_counts', 'leiden', 'modified_cell_type', 'IFN_stim', 'Activation', 'Exhaust', 'Mem', 'state', 'is_2D2'
      var:	'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
      uns:	'hvg', 'is_2D2_colors', 'leiden', 'leiden_colors', 'log1p', 'modified_cell_type_colors', 'mouse_id_colors', 'neighbors', 'pca', 'state_colors', 'tissue_colors', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs'
      obsp:	'connectivities', 'distances'
    airr:	77500 x 0
      obs:	'receptor_type', 'receptor_subtype', 'chain_pairing', 'clone_id', 'clone_id_size', 'clonal_expansion'
      uns:	'chain_indices', 'clone_id', 'ir_dist_nt_identity'
      obsm:	'airr', 'chain_indices'

In [97]:
TCR_emb = pd.read_csv('/ihome/ylee/yiz133/Code/deepTCR/unSupervised_cluster/DTCRU_features_all.csv')
TCR_emb['CDR3_Beta'] = TCR_emb['CDR3_Beta'].str[1:-1]
TCR_emb = TCR_emb.rename(columns=lambda c: f"Temb_{c}" if str(c).isdigit() else c)
TCR_emb.head(5)

,CDR3_Beta,V_Beta,J_Beta,Label,Temb_0,Temb_1,Temb_2,Temb_3,Temb_4,Temb_5,...,Temb_86,Temb_87,Temb_88,Temb_89,Temb_90,Temb_91,Temb_92,Temb_93,Temb_94,Temb_95
0,ASSQDGDSDYT,TRBV5,TRBJ1-2,all\airr_test.tsv,1.927215,-0.325208,-1.289598,-0.667583,1.343599,1.640880,...,1.654847,-1.277153,0.012371,-0.836811,-0.961512,-0.927956,0.199898,-0.808558,0.170265,0.556819
1,TCSADPGVNQDTQY,TRBV1,TRBJ2-5,all\airr_test.tsv,1.239994,-0.253389,0.172573,-2.288065,-1.026347,0.161365,...,-0.529986,-0.898209,-0.641696,0.193812,0.772396,0.155341,-0.170177,0.015360,0.482544,1.344052
2,ASGEDTQY,TRBV12-2,TRBJ2-5,all\airr_test.tsv,0.647911,-0.310563,-0.751777,-2.075442,0.954144,0.476126,...,-0.000281,-0.646268,-0.195188,-0.561911,0.343529,-1.132762,0.214265,-0.558531,0.125088,0.402434
3,ASGETGGNYAEQF,TRBV12-2,TRBJ2-1,all\airr_test.tsv,0.882616,-2.164111,-1.264558,-0.301007,0.867764,-2.040081,...,-1.105132,0.631774,0.814376,-0.178991,0.031161,0.303922,0.147796,-0.356257,0.849508,0.329896
4,ASSPDRDPGNTLY,TRBV19,TRBJ1-3,all\airr_test.tsv,-2.617389,-0.173138,-0.049102,0.601537,-0.175316,0.291019,...,-0.842881,0.273394,-1.606196,1.072606,-0.537985,1.417173,0.392797,0.080694,0.673055,-1.171613


In [98]:
obs_features = pd.concat(
    [
    mdata.obs[['TCRdist_MOG', 'in_two_tissue']],
    mdata['gex'].obs[['mouse_id', 'tissue', 'modified_cell_type']],
    mdata['airr'].obs[['clone_id_size']]
], axis=1)

obs_features = obs_features[~obs_features['modified_cell_type'].isin(['Myloid','NaN'])]

obs_features

,TCRdist_MOG,in_two_tissue,mouse_id,tissue,modified_cell_type,clone_id_size
AAACCAAAGGGGAGCT-1_0516_CNS,75.0,False,5_3,CN,CD4+ T,1
AAACCAGCACGTAAAG-1_0516_CNS,68.0,False,5_3,CN,CD8+ T,1
AAACCCATCAGTATCG-1_0516_CNS,123.0,False,5_3,CN,Treg,1
AAACCGTGTATCAGCG-1_0516_CNS,80.0,True,5_4,CN,CD8+ T,7
AAACCTCCAAAGCTTC-1_0516_CNS,99.0,True,5_4,CN,CD8+ T,21
...,...,...,...,...,...,...
GTTGTGGGTGGAGCGA-1_0605_SPL,48.0,False,6_1,SP,CD8+ T,1
GTTGTGGGTGGATCGC-1_0605_SPL,95.0,False,6_4,SP,CD8+ T,1
GTTGTGGGTGGCTGTA-1_0605_SPL,84.0,False,6_1,SP,CD8+ T,1
GTTGTGGGTGTCCCTG-1_0605_SPL,101.0,False,6_2,SP,Treg,1


In [99]:
type_markers = ['Cd4', 'Cd8a','Cd8b1','Nkg7','Foxp3', 'Ikzf2', 'Ctla4', 'Il2ra','Ccr6', 'Il22']
state_markers = ['Isg15', 'Gbp2', 'Ifih1', 'Icos', 'Cd69', 'Cd28', 'Pdcd1', 'Lag3', 'Havcr2', 'Ccr7', 'Sell', 'Cd27']

gene_features = mdata['gex'][:, type_markers].to_df()  
gene_features = gene_features.add_prefix("gene_")
gene_features

,gene_Cd4,gene_Cd8a,gene_Cd8b1,gene_Nkg7,gene_Foxp3,gene_Ikzf2,gene_Ctla4,gene_Il2ra,gene_Ccr6,gene_Il22
AAACCAAAGGGGAGCT-1_0516_CNS,0.765119,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
AAACCAGCACGTAAAG-1_0516_CNS,1.789832,0.000000,0.000000,0.000000,0.000000,0.000000,0.484619,0.000000,0.0,2.645797
AAACCATTCCTCCGGT-1_0516_CNS,0.868464,0.000000,0.000000,2.490389,0.000000,0.000000,0.868464,0.000000,0.0,0.000000
AAACCCATCAGTATCG-1_0516_CNS,1.432564,0.000000,0.000000,0.000000,2.416465,2.229209,2.471826,1.432564,0.0,0.000000
AAACCCCAGCCTAAGC-1_0516_CNS,0.853588,0.000000,0.000000,3.229421,0.000000,0.853588,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
GTTGTGGGTGGAGCGA-1_0605_SPL,2.206581,0.000000,1.105638,0.000000,0.000000,1.105638,1.105638,1.617863,0.0,0.000000
GTTGTGGGTGGATCGC-1_0605_SPL,0.000000,1.923915,1.205697,1.736629,0.774516,0.000000,0.000000,0.000000,0.0,0.000000
GTTGTGGGTGGCTGTA-1_0605_SPL,0.000000,1.290984,2.187072,1.836211,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
GTTGTGGGTGTCCCTG-1_0605_SPL,1.646059,0.873592,0.000000,0.000000,0.000000,0.000000,0.873592,0.000000,0.0,0.000000


In [100]:
obs_features = obs_features.merge(gene_features, left_index=True, right_index=True, how='')
obs_features

,TCRdist_MOG,in_two_tissue,mouse_id,tissue,modified_cell_type,clone_id_size,gene_Cd4,gene_Cd8a,gene_Cd8b1,gene_Nkg7,gene_Foxp3,gene_Ikzf2,gene_Ctla4,gene_Il2ra,gene_Ccr6,gene_Il22
AAACCAAAGGGGAGCT-1_0516_CNS,75.0,False,5_3,CN,CD4+ T,1,0.765119,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AAACCAGCACGTAAAG-1_0516_CNS,68.0,False,5_3,CN,CD8+ T,1,1.789832,0.000000,0.000000,0.000000,0.000000,0.000000,0.484619,0.000000,0.000000,2.645797
AAACCCATCAGTATCG-1_0516_CNS,123.0,False,5_3,CN,Treg,1,1.432564,0.000000,0.000000,0.000000,2.416465,2.229209,2.471826,1.432564,0.000000,0.000000
AAACCGTGTATCAGCG-1_0516_CNS,80.0,True,5_4,CN,CD8+ T,7,1.511135,0.000000,0.000000,2.087378,0.000000,0.000000,1.137552,0.880767,0.000000,0.000000
AAACCTCCAAAGCTTC-1_0516_CNS,99.0,True,5_4,CN,CD8+ T,21,0.660404,0.000000,0.660404,0.000000,1.054712,1.736576,1.996318,1.889104,2.021417,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTTGTGGGTGGAGCGA-1_0605_SPL,48.0,False,6_1,SP,CD8+ T,1,2.206581,0.000000,1.105638,0.000000,0.000000,1.105638,1.105638,1.617863,0.000000,0.000000
GTTGTGGGTGGATCGC-1_0605_SPL,95.0,False,6_4,SP,CD8+ T,1,0.000000,1.923915,1.205697,1.736629,0.774516,0.000000,0.000000,0.000000,0.000000,0.000000
GTTGTGGGTGGCTGTA-1_0605_SPL,84.0,False,6_1,SP,CD8+ T,1,0.000000,1.290984,2.187072,1.836211,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
GTTGTGGGTGTCCCTG-1_0605_SPL,101.0,False,6_2,SP,Treg,1,1.646059,0.873592,0.000000,0.000000,0.000000,0.000000,0.873592,0.000000,0.000000,0.000000


In [101]:
TCRs = mdata.obs[['VDJ_1_cdr3_aa', 'VDJ_1_v_call', 'VDJ_1_j_call']]

TCRs['VDJ_1_v_call'] = TCRs['VDJ_1_v_call'].str.split('+').str[0]
TCRs['VDJ_1_j_call'] = TCRs['VDJ_1_j_call'].str.split('+').str[0]

TCRs['BC'] = TCRs.index

TCRs_combined = TCRs.merge(
    TCR_emb,
    left_on=['VDJ_1_cdr3_aa', 'VDJ_1_v_call', 'VDJ_1_j_call'],
    right_on=['CDR3_Beta', 'V_Beta', 'J_Beta'],
    how='inner')

TCRs_combined.dropna(inplace=True)

TCRs_combined.set_index(['BC'],inplace=True)
TCRs_emb = TCRs_combined.drop(columns=['VDJ_1_cdr3_aa','VDJ_1_v_call','VDJ_1_j_call','CDR3_Beta','V_Beta','J_Beta', 'Label'])
TCRs_emb

,Temb_0,Temb_1,Temb_2,Temb_3,Temb_4,Temb_5,Temb_6,Temb_7,Temb_8,Temb_9,...,Temb_86,Temb_87,Temb_88,Temb_89,Temb_90,Temb_91,Temb_92,Temb_93,Temb_94,Temb_95
BC,,,,,,,,,,,,,,,,,,,,,
AAACCAAAGGGGAGCT-1_0516_CNS,-1.074540,-1.381444,2.003792,1.898528,0.110185,-0.030307,0.024259,0.281583,1.653965,-1.235007,...,-0.788144,0.851781,1.434349,0.478742,-0.463182,1.459020,-0.488335,0.387675,0.094444,0.596310
AAACCAGCACGTAAAG-1_0516_CNS,2.608337,-0.347479,-1.064834,1.058512,0.823411,1.060818,0.594391,0.297602,-1.839484,0.569416,...,0.942622,-0.494397,0.797757,-0.827705,0.084715,-0.661331,-1.007851,0.356426,-0.313030,0.784515
AAACCATTCCTCCGGT-1_0516_CNS,1.678821,-0.290928,-1.651523,0.081453,0.249506,-0.952036,-0.314082,-0.220502,0.118954,-1.374099,...,1.476976,-0.958903,0.388932,-1.236487,-1.161050,0.453646,0.438231,-0.590204,-0.502968,0.574626
AATGGAGGTATCAATG-1_0516_CNS,1.678821,-0.290928,-1.651523,0.081453,0.249506,-0.952036,-0.314082,-0.220502,0.118954,-1.374099,...,1.476976,-0.958903,0.388932,-1.236487,-1.161050,0.453646,0.438231,-0.590204,-0.502968,0.574626
ACCTCCATCGTACCTT-1_0516_CNS,1.678821,-0.290928,-1.651523,0.081453,0.249506,-0.952036,-0.314082,-0.220502,0.118954,-1.374099,...,1.476976,-0.958903,0.388932,-1.236487,-1.161050,0.453646,0.438231,-0.590204,-0.502968,0.574626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTTGTGGGTACTACGG-1_0605_SPL,0.399106,1.107546,1.753217,0.516664,-1.959969,0.645131,-1.748370,0.478381,1.224798,0.813876,...,-0.601751,0.291710,0.288384,-0.211543,0.875779,0.282724,-0.290949,1.583146,-0.000488,0.585590
GTTGTGGGTAGCGTAT-1_0605_SPL,0.952979,1.531799,0.260634,1.549651,-1.820541,-2.369582,0.580685,2.985489,-0.504145,-0.981985,...,-0.016571,0.560196,0.787294,-0.269766,0.290741,-0.114308,-1.337781,0.908350,-0.450412,0.344480
GTTGTGGGTGGATCGC-1_0605_SPL,0.834183,2.158250,-1.592759,2.809128,1.578484,1.096320,1.621659,0.978207,-1.022636,0.884884,...,1.282941,-0.365931,-0.604710,-0.426701,-0.451893,0.027583,-0.175119,-0.080719,-1.204319,-0.843467


In [102]:
DF_all = obs_features.merge(TCRs_emb,  left_index=True, right_index=True, how='inner')
DF_all

,TCRdist_MOG,in_two_tissue,mouse_id,tissue,modified_cell_type,clone_id_size,gene_Cd4,gene_Cd8a,gene_Cd8b1,gene_Nkg7,...,Temb_86,Temb_87,Temb_88,Temb_89,Temb_90,Temb_91,Temb_92,Temb_93,Temb_94,Temb_95
AAACCAAAGGGGAGCT-1_0516_CNS,75.0,False,5_3,CN,CD4+ T,1,0.765119,0.000000,0.000000,0.000000,...,-0.788144,0.851781,1.434349,0.478742,-0.463182,1.459020,-0.488335,0.387675,0.094444,0.596310
AAACCAGCACGTAAAG-1_0516_CNS,68.0,False,5_3,CN,CD8+ T,1,1.789832,0.000000,0.000000,0.000000,...,0.942622,-0.494397,0.797757,-0.827705,0.084715,-0.661331,-1.007851,0.356426,-0.313030,0.784515
AAACCCATCAGTATCG-1_0516_CNS,123.0,False,5_3,CN,Treg,1,1.432564,0.000000,0.000000,0.000000,...,0.473147,-0.169886,0.128934,-0.096707,0.996795,-0.218550,0.101811,0.576915,0.367889,1.286326
AAACCGTGTATCAGCG-1_0516_CNS,80.0,True,5_4,CN,CD8+ T,7,1.511135,0.000000,0.000000,2.087378,...,-0.484908,-0.282734,1.543997,-0.155556,0.390656,-0.580605,-1.240201,0.596745,-0.158559,0.625626
AAACCTCCAAAGCTTC-1_0516_CNS,99.0,True,5_4,CN,CD8+ T,21,0.660404,0.000000,0.660404,0.000000,...,-0.037155,-0.595106,0.503946,-1.865435,-1.142989,-0.218842,0.953885,-0.124080,0.306239,-0.699992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTTGTGGGTAGCGTAT-1_0605_SPL,80.0,False,6_4,SP,CD4+ T,1,1.081856,0.000000,1.081856,0.000000,...,-0.016571,0.560196,0.787294,-0.269766,0.290741,-0.114308,-1.337781,0.908350,-0.450412,0.344480
GTTGTGGGTGGAGCGA-1_0605_SPL,48.0,False,6_1,SP,CD8+ T,1,2.206581,0.000000,1.105638,0.000000,...,1.387276,0.287732,-0.488168,0.796769,0.294418,-1.140490,1.266609,-0.937512,-0.239104,0.003485
GTTGTGGGTGGATCGC-1_0605_SPL,95.0,False,6_4,SP,CD8+ T,1,0.000000,1.923915,1.205697,1.736629,...,1.282941,-0.365931,-0.604710,-0.426701,-0.451893,0.027583,-0.175119,-0.080719,-1.204319,-0.843467
GTTGTGGGTGGCTGTA-1_0605_SPL,84.0,False,6_1,SP,CD8+ T,1,0.000000,1.290984,2.187072,1.836211,...,0.313365,-0.865876,-0.766232,0.641368,0.322743,-0.935293,-0.510938,-0.473883,-0.412163,-0.113040


In [103]:
DF_all.to_csv('train_')